# Яндекс.Музыка

Сравнение Москвы и Петербурга окружено мифами. Например:
 * Москва — мегаполис, подчинённый жёсткому ритму рабочей недели;
 * Петербург — культурная столица, со своими вкусами.

На данных Яндекс.Музыки вы сравните поведение пользователей двух столиц.

**Цель исследования** — проверьте три гипотезы:
1. Активность пользователей зависит от дня недели. Причём в Москве и Петербурге это проявляется по-разному.
2. В понедельник утром в Москве преобладают одни жанры, а в Петербурге — другие. Так же и вечером пятницы преобладают разные жанры — в зависимости от города. 
3. Москва и Петербург предпочитают разные жанры музыки. В Москве чаще слушают поп-музыку, в Петербурге — русский рэп.

**Ход исследования**

Данные о поведении пользователей вы получите из файла `yandex_music_project.csv`. О качестве данных ничего не известно. Поэтому перед проверкой гипотез понадобится обзор данных. 

Вы проверите данные на ошибки и оцените их влияние на исследование. Затем, на этапе предобработки вы поищете возможность исправить самые критичные ошибки данных.
 
Таким образом, исследование пройдёт в три этапа:
 1. Обзор данных.
 2. Предобработка данных.
 3. Проверка гипотез.



## Обзор данных

Составим первое представление о данных Яндекс.Музыки.




In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('/datasets/yandex_music_project.csv')

Посмотрим на первые десять записей таблицы:

In [ ]:
df.head(10)

Посмотрим общую информацию о таблице:

In [ ]:
df.info() 

Итак, в таблице семь столбцов. Тип данных во всех столбцах — `object`.

Согласно документации к данным:
* `userID` — идентификатор пользователя;
* `Track` — название трека;  
* `artist` — имя исполнителя;
* `genre` — название жанра;
* `City` — город пользователя;
* `time` — время начала прослушивания;
* `Day` — день недели.

В названиях колонок видны три нарушения стиля:
1. Строчные буквы сочетаются с прописными.
2. Встречаются пробелы.
3. Количество значений в столбцах различается. Значит, в данных есть пропущенные значения. Присутствует "верблюжий" регистр, его мы заменим на "змеиный"





**Выводы**

В каждой строке таблицы — данные о прослушанном треке. Часть колонок описывает саму композицию: название, исполнителя и жанр. Остальные данные рассказывают о пользователе: из какого он города, когда он слушал музыку. 

Предварительно можно утверждать, что, данных достаточно для проверки гипотез. Но встречаются пропуски в данных, а в названиях колонок — расхождения с хорошим стилем.

Чтобы двигаться дальше, нужно устранить проблемы в данных.

## Предобработка данных
Исправим стиль в заголовках столбцов, исключим пропуски. Затем проверим данные на дубликаты.

### Стиль заголовков

In [ ]:
df.columns 

Приведем названия в соответствие с хорошим стилем:
* несколько слов в названии запишем в «змеином_регистре»,
* все символы сделаем строчными,
* устраним пробелы.

Для этого переименуем колонки так:
* `'  userID'` → `'user_id'`;
* `'Track'` → `'track'`;
* `'  City  '` → `'city'`;
* `'Day'` → `'day'`.

In [ ]:
df = df.rename(columns = {
    '  userID':'user_id', 
    'Track' : 'track', 
    '  City  ' : 'city',
    'Day' : 'day'}) 

Проверьте результат. Для этого ещё раз выведите на экран названия столбцов:

In [ ]:
df.columns 

### Пропуски значений
Сначала посчитаем, сколько в таблице пропущенных значений. 

In [ ]:
df.isna().sum() 

Не все пропущенные значения влияют на исследование. Так в `track` и `artist` пропуски не важны для нашей работы. Достаточно заменить их явными обозначениями.

Но пропуски в `genre` могут помешать сравнению музыкальных вкусов в Москве и Санкт-Петербурге. На практике было бы правильно установить причину пропусков и восстановить данные. Такой возможности нет в учебном проекте. Придётся:
* заполнить и эти пропуски явными обозначениями,
* оценить, насколько они повредят расчётам. 

Заменим пропущенные значения в столбцах `track`, `artist` и `genre` на строку `'unknown'`. Для этого создадим список `columns_to_replace`, переберем его элементы циклом `for` и для каждого столбца выполним замену пропущенных значений:

In [ ]:
columns_to_replace = ['track', 'artist', 'genre']
for index in columns_to_replace:
    df[index] = df[index].fillna('unknown')
    
    # перебор названий столбцов в цикле и замена пропущенных значений на 'unknown'

Убедимся, что в таблице не осталось пропусков.

In [ ]:
df.isna().sum() 

### Дубликаты
Посчитаем явные дубликаты в таблице

In [ ]:
df.duplicated().sum() 

Удалим явные дубликаты

In [ ]:
df = df.drop_duplicates().reset_index(drop=True)


In [ ]:
df.duplicated().sum() 

Теперь избавимся от неявных дубликатов в колонке `genre`. Например, название одного и того же жанра может быть записано немного по-разному. Такие ошибки тоже повлияют на результат исследования.

Выведем на экран список уникальных названий жанров, отсортированный в алфавитном порядке. 


In [ ]:
df['genre'].sort_values().unique() 

Видим следующие неявные дубликаты:
* *hip*,
* *hop*,
* *hip-hop*.

Чтобы очистить от них таблицу, напишите функцию `replace_wrong_genres()` с двумя параметрами: 
* `wrong_genres` — список дубликатов,
* `correct_genre` — строка с правильным значением.


In [ ]:
wrong_genres = ['hip', 'hop', 'hip-hop']
correct_genre = 'hiphop'
def replace_wrong_genres(wrong_genres, correct_genre):
    df['genre'] = df['genre'].replace(wrong_genres, correct_genre)



In [ ]:
replace_wrong_genres(wrong_genres, correct_genre) # Устранение неявных дубликатов

In [ ]:
df['genre'].sort_values().unique()# Проверка на неявные дубликаты

**Выводы**

Предобработка обнаружила три проблемы в данных:

- нарушения в стиле заголовков,
- пропущенные значения,
- дубликаты — явные и неявные.

Исправлены заголовки, чтобы упростить работу с таблицей. Без дубликатов исследование станет более точным.

Пропущенные значения заменены на `'unknown'`. Ещё предстоит увидеть, не повредят ли исследованию пропуски в колонке `genre`.

Теперь можно перейти к проверке гипотез. 

## Проверка гипотез

### Сравнение поведения пользователей двух столиц

Первая гипотеза утверждает, что пользователи по-разному слушают музыку в Москве и Санкт-Петербурге. Проверим это предположение по данным о трёх днях недели — понедельнике, среде и пятнице. Для этого:

* Разделим пользователей Москвы и Санкт-Петербурга
* Сравним, сколько треков послушала каждая группа пользователей в понедельник, среду и пятницу.


In [ ]:
city_grouping = df.groupby('city')['time'].count()
city_grouping

# Подсчёт прослушиваний в каждом городе

В Москве прослушиваний больше, чем в Петербурге. Из этого не следует, что московские пользователи чаще слушают музыку. Просто самих пользователей в Москве больше.

Теперь сгруппируем данные по дню недели и подсчитаем прослушивания в понедельник, среду и пятницу. 

In [ ]:
day_grouping = df.groupby('day')['time'].count()
day_grouping
# Подсчёт прослушиваний в каждый из трёх дней

В среднем пользователи из двух городов менее активны по средам. Но картина может измениться, если рассмотреть каждый город в отдельности.



Создайдим функцию `number_tracks()`, которая посчитает прослушивания для заданного дня и города. 


In [ ]:
def number_tracks(day, city):
    track_list = df[(df['day'] == day) & (df['city'] ==city)]
    track_list_count = track_list['user_id'].count()
    return track_list_count                

# <создание функции number_tracks()>
# Объявляется функция с двумя параметрами: day, city.
# В переменной track_list сохраняются те строки таблицы df, для которых 
# значение в столбце 'day' равно параметру day и одновременно значение
# в столбце 'city' равно параметру city (используйте последовательную фильтрацию
# с помощью логической индексации).
# В переменной track_list_count сохраняется число значений столбца 'user_id',
# рассчитанное методом count() для таблицы track_list.
# Функция возвращает число - значение track_list_count.

# Функция для подсчёта прослушиваний для конкретного города и дня.
# С помощью последовательной фильтрации с логической индексацией она 
# сначала получит из исходной таблицы строки с нужным днём,
# затем из результата отфильтрует строки с нужным городом,
# методом count() посчитает количество значений в колонке user_id. 
# Это количество функция вернёт в качестве результата

In [ ]:
number_tracks('Monday', 'Moscow') # количество прослушиваний в Москве по понедельникам

In [ ]:
number_tracks('Monday', 'Saint-Petersburg')# количество прослушиваний в Санкт-Петербурге по понедельникам

In [ ]:
number_tracks('Wednesday', 'Moscow')# количество прослушиваний в Москве по средам

In [ ]:
number_tracks('Wednesday', 'Saint-Petersburg') # количество прослушиваний в Санкт-Петербурге по средам

In [ ]:
number_tracks('Friday', 'Moscow')# количество прослушиваний в Москве по пятницам

In [ ]:
number_tracks('Friday', 'Saint-Petersburg')# количество прослушиваний в Санкт-Петербурге по пятницам

Создадим c помощью конструктора `pd.DataFrame` таблицу, где
* названия колонок — `['city', 'monday', 'wednesday', 'friday']`;
* данные — результаты, которые вы получили с помощью `number_tracks`.

In [ ]:
data = [
    [
        'Moscow',
        number_tracks('Monday', 'Moscow'),
        number_tracks('Wednesday', 'Moscow'),
        number_tracks('Friday', 'Moscow')
    ],
    [
        'Saint-Petersburg',
        number_tracks('Monday', 'Saint-Petersburg'),
        number_tracks('Wednesday', 'Saint-Petersburg'),
        number_tracks('Friday', 'Saint-Petersburg')
    ]
]

results_table = pd.DataFrame(data=data, columns=['city', 'monday', 'wednesday', 'friday']) # Таблица с результатамиresults_table = pd.DataFrame(data=[['Moscow',15740, 5614, , 7003, 15945, 5895]] , columns=['city', 'monday', 'wednesday', 'friday']) # Таблица с результатами
results_table

**Выводы**

Данные показывают разницу поведения пользователей:

- В Москве пик прослушиваний приходится на понедельник и пятницу, а в среду заметен спад.
- В Петербурге, наоборот, больше слушают музыку по средам. Активность в понедельник и пятницу здесь почти в равной мере уступает среде.

Значит, данные говорят в пользу первой гипотезы.

### Музыка в начале и в конце недели

Согласно второй гипотезе, утром в понедельник в Москве преобладают одни жанры, а в Петербурге — другие. Так же и вечером пятницы преобладают разные жанры — в зависимости от города.

Сохраним таблицы с данными в две переменные:
* по Москве — в `moscow_general`;
* по Санкт-Петербургу — в `spb_general`.

In [ ]:
moscow_general = df[(df['city'] == 'Moscow')]
moscow_general



In [ ]:
spb_general = df[(df['city'] == 'Saint-Petersburg')]
spb_general


Создадим функцию `genre_weekday()` с четырьмя параметрами:
* таблица (датафрейм) с данными,
* день недели,
* начальная временная метка в формате 'hh:mm', 
* последняя временная метка в формате 'hh:mm'.

Функция возвращает информацию о топ-10 жанров тех треков, которые прослушивали в указанный день, в промежутке между двумя отметками времени.

In [ ]:
def genre_weekday(table, day, time1, time2):
    genre_df = table[(table['day'] == day) & (table['time'] > time1) & (table['time'] < time2)]
    genre_df_count = genre_df.groupby('genre')['time'].count()
    genre_df_sorted = genre_df_count.sort_values(ascending=False)
    return genre_df_sorted.head(10)



Cравним результаты функции `genre_weekday()` для Москвы и Санкт-Петербурга в понедельник утром (с 7:00 до 11:00) и в пятницу вечером (с 17:00 до 23:00):

In [ ]:
genre_weekday(moscow_general, 'Monday', '07:00:00', '11:00:00')


In [ ]:
genre_weekday(spb_general, 'Monday', '07:00:00', '11:00:00')


In [ ]:
genre_weekday(moscow_general, 'Friday', '17:00:00', '23:00:00')


In [ ]:
genre_weekday(spb_general, 'Friday', '17:00:00', '23:00:00')


**Выводы**

Если сравнить топ-10 жанров в понедельник утром, можно сделать такие выводы:

1. В Москве и Петербурге слушают похожую музыку. Единственное отличие — в московский рейтинг вошёл жанр “world”, а в петербургский — джаз и классика.

2. В Москве пропущенных значений оказалось так много, что значение `'unknown'` заняло десятое место среди самых популярных жанров. Значит, пропущенные значения занимают существенную долю в данных и угрожают достоверности исследования.

Вечер пятницы не меняет эту картину. Некоторые жанры поднимаются немного выше, другие спускаются, но в целом топ-10 остаётся тем же самым.

Таким образом, вторая гипотеза подтвердилась лишь частично:
* Пользователи слушают похожую музыку в начале недели и в конце.
* Разница между Москвой и Петербургом не слишком выражена. В Москве чаще слушают русскую популярную музыку, в Петербурге — джаз.

Однако пропуски в данных ставят под сомнение этот результат. В Москве их так много, что рейтинг топ-10 мог бы выглядеть иначе, если бы не утерянные  данные о жанрах.

### Жанровые предпочтения в Москве и Петербурге

Гипотеза: Петербург — столица рэпа, музыку этого жанра там слушают чаще, чем в Москве.  А Москва — город контрастов, в котором, тем не менее, преобладает поп-музыка.

Сгруппируем таблицу `moscow_general` по жанру и посчитаем прослушивания треков каждого жанра методом `count()`. Затем отсортируем результат в порядке убывания и сохраним его в таблице `moscow_genres`.

In [ ]:
group_moscow_general = moscow_general.groupby('genre')['time'].count()
moscow_genres = group_moscow_general.sort_values(ascending=False)


In [ ]:
moscow_genres.head(10)

Теперь повторим то же и для Петербурга.



In [ ]:
group_spb_general = spb_general.groupby('genre')['time'].count()
spb_genres = group_spb_general.sort_values(ascending=False)
# одной строкой: группировка таблицы spb_general по столбцу 'genre', 
# подсчёт числа значений 'genre' в этой группировке методом count(), 
# сортировка получившегося Series в порядке убывания и сохранение в spb_genres

Выведите на экран первые десять строк `spb_genres`:

In [ ]:
spb_genres.head(10)

**Выводы**

Гипотеза частично подтвердилась:
* Поп-музыка — самый популярный жанр в Москве, как и предполагала гипотеза. Более того, в топ-10 жанров встречается близкий жанр — русская популярная музыка.
* Вопреки ожиданиям, рэп одинаково популярен в Москве и Петербурге. 


## Итоги исследования

Вы проверили три гипотезы и установили:

1. День недели по-разному влияет на активность пользователей в Москве и Петербурге. 

Первая гипотеза полностью подтвердилась.

2. Музыкальные предпочтения не сильно меняются в течение недели — будь то Москва или Петербург. Небольшие различия заметны в начале недели, по понедельникам:
* в Москве слушают музыку жанра “world”,
* в Петербурге — джаз и классику.

Таким образом, вторая гипотеза подтвердилась лишь отчасти. Этот результат мог оказаться иным, если бы не пропуски в данных.

3. Во вкусах пользователей Москвы и Петербурга больше общего чем различий. Вопреки ожиданиям, предпочтения жанров в Петербурге напоминают московские.

Третья гипотеза не подтвердилась. Если различия в предпочтениях и существуют, на основной массе пользователей они незаметны.

**На практике исследования содержат проверки статистических гипотез.**
Из данных одного сервиса не всегда можно сделать вывод о всех жителях города.
Проверки статистических гипотез покажут, насколько они достоверны, исходя из имеющихся данных. 
С методами проверок гипотез вы ещё познакомитесь в следующих темах.